In [1]:
!nvidia-smi

Tue Sep 17 14:57:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Supercomp - Atividade 10

## Exemplos

EXEMPLO 1

In [2]:
%%writefile exemplo1.cu
#include <thrust/device_vector.h> // Inclui a biblioteca para vetores na GPU (device)
#include <thrust/host_vector.h>   // Inclui a biblioteca para vetores na CPU (host)
#include <iostream>               // Biblioteca padrão para entrada e saída de dados

int main() {
    // Cria um vetor na CPU (host) com 5 elementos, todos inicializados com 0
    thrust::host_vector<double> host(5, 0);
    host[4] = 35; // Altera o último elemento do vetor para 35

    /* Os dados do vetor 'host' são copiados para a GPU,
       criando um vetor equivalente na GPU (device) */
    thrust::device_vector<double> dev(host);

    /* Altera o vetor na CPU, mas não afeta o vetor na GPU,
       pois a cópia já foi feita anteriormente */
    host[2] = 12; // Altera o terceiro elemento do vetor na CPU para 12

    // Exibe os elementos do vetor na CPU
    printf("Host vector: ");
    for (auto i = host.begin(); i != host.end(); i++) {
        std::cout << *i << " "; // Acesso rápido aos elementos na CPU
    }
    printf("\n");

    // Exibe os elementos do vetor na GPU
    printf("Device vector: ");
    for (auto i = dev.begin(); i != dev.end(); i++) {
        std::cout << *i << " "; // Acesso aos elementos na GPU é lento, pois os dados estão na GPU
    }
    printf("\n");
}

Writing exemplo1.cu


In [3]:
!nvcc -arch=sm_75 -std=c++14 exemplo1.cu -o exemplo1

In [4]:
!./exemplo1

Host vector: 0 0 12 0 35 
Device vector: 0 0 0 0 35 


EXEMPLO 2

In [5]:
%%writefile exemplo2.cu
#include <thrust/device_vector.h>              // Inclui a biblioteca para vetores na GPU
#include <thrust/host_vector.h>                // Inclui a biblioteca para vetores na CPU
#include <thrust/sequence.h>                   // Inclui a função para preencher vetores com uma sequência de números
#include <thrust/functional.h>                 // Inclui operações matemáticas padrão, como soma e multiplicação
#include <thrust/transform.h>                  // Inclui a função para transformar vetores com operações ponto a ponto
#include <thrust/iterator/constant_iterator.h> // Inclui o iterador constante necessário
#include <iostream>

int main() {
    // Cria um vetor na GPU (device_vector) com 10 elementos, todos inicializados com 0
    thrust::device_vector<double> V1(10, 0);
    // Preenche V1 com uma sequência de números: {0, 1, 2, ..., 9}
    thrust::sequence(V1.begin(), V1.end());

    // Cria um vetor na GPU com 5 elementos, todos inicializados com 0
    thrust::device_vector<double> V2(5, 0);
    // Preenche os dois primeiros elementos de V2 com 5.5: {5.5, 5.5, 0, 0, 0}
    thrust::fill(V2.begin(), V2.begin() + 2, 5.5);
    // Preenche os elementos restantes de V2 com 10: {5.5, 5.5, 10, 10, 10}
    thrust::fill(V2.begin() + 2, V2.end(), 10);

    // Cria dois vetores na GPU com 10 elementos, para armazenar resultados das operações
    thrust::device_vector<double> V3(10); // Vetor para armazenar o resultado da soma
    thrust::device_vector<double> V4(10); // Vetor para armazenar o resultado da multiplicação

    // Aplica a operação de soma elemento por elemento entre V1 e V2 e armazena o resultado em V3
    // Como V2 tem menos elementos, o restante de V1 é somado com zeros (elemento padrão).
    // Resultado: V3 = {0+5.5, 1+5.5, 2+10, 3+10, 4+10, 5+0, ..., 9+0}
    thrust::transform(V1.begin(), V1.end(), V2.begin(), V3.begin(), thrust::plus<double>());

    // Aplica multiplicação de V1 com o valor constante 0.5
    // e armazenaria o resultado em V4.
    thrust::transform(V1.begin(), V1.end(), thrust::constant_iterator<double>(0.5), V4.begin(), thrust::multiplies<double>());

    // Imprime os elementos de V1
    printf("V1: ");
    for (thrust::device_vector<double>::iterator i = V1.begin(); i != V1.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V1
    }
    printf("\n");

    // Imprime os elementos de V2
    printf("V2: ");
    for (thrust::device_vector<double>::iterator i = V2.begin(); i != V2.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V2
    }
    printf("\n");

    // Imprime os elementos de V3
    printf("V3: ");
    for (thrust::device_vector<double>::iterator i = V3.begin(); i != V3.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V3
    }
    printf("\n");

    // Imprime o vetor V4, que foi criado mas não utilizado.
     printf("V4: ");
    for (thrust::device_vector<double>::iterator i = V4.begin(); i != V4.end(); i++) {
         std::cout << *i << " ";
    }
    printf("\n");

    return 0;
}

Writing exemplo2.cu


In [6]:
!nvcc -arch=sm_75 -std=c++14 exemplo2.cu -o exemplo2

In [7]:
!./exemplo2

V1: 0 1 2 3 4 5 6 7 8 9 
V2: 5.5 5.5 10 10 10 
V3: 5.5 6.5 12 13 14 5 6 7 8 9 
V4: 0 0.5 1 1.5 2 2.5 3 3.5 4 4.5 


STOCKS

In [8]:
%%writefile stocks.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;

int main() {
    int n = 2518;                   // Define o tamanho do vetor (2518 elementos)
    double value = 0.0;             // Variável auxiliar para armazenar o valor lido da entrada
    std::chrono::duration<double> diff; // Variável para armazenar a diferença de tempo calculada

    // Marca o início da medição do tempo de leitura dos dados
    auto leitura_i = std::chrono::steady_clock::now();

    // Cria um vetor na CPU (host_vector) com n elementos, todos inicializados com 0
    thrust::host_vector<double> host(n, 0);

    // Loop para ler n valores da entrada padrão e armazená-los no vetor host
    for (int i = 0; i < n; i++) {
        cin >> value;              // Lê um valor da entrada padrão
        host[i] = value;           // Armazena o valor lido no vetor host
    }

    // Marca o fim da medição do tempo de leitura dos dados
    auto leitura_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na leitura dos dados
    diff = leitura_f - leitura_i;

    // Exibe o tempo de leitura em segundos
    cout << "Tempo de LEITURA (em segundos)  " << diff.count() << endl;

    // Marca o início da medição do tempo de cópia dos dados da CPU para a GPU
    auto copia_i = std::chrono::steady_clock::now();

    // Cria um vetor na GPU (device_vector) copiando os dados do vetor host da CPU
    thrust::device_vector<double> dev(host);

    // Marca o fim da medição do tempo de cópia dos dados
    auto copia_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na cópia dos dados para a GPU
    diff = copia_f - copia_i;

    // Exibe o tempo de cópia em segundos
    cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << endl;

    return 0;
}

Writing stocks.cu


In [9]:
!nvcc -arch=sm_70 -std=c++14 stocks.cu -o stocks

In [10]:
!./stocks < stocks-google.txt

Tempo de LEITURA (em segundos)  0.00228922
Tempo de CÓPIA (em segundos)  0.137235


SOMA

In [11]:
%%writefile soma.cu
#include <thrust/device_vector.h>
#include <thrust/reduce.h>
#include <iostream>

int main() {
    // Criando um vetor na GPU com 5 elementos: {10, 20, 30, 40, 50}
    thrust::device_vector<int> vec_gpu(5);
    thrust::sequence(vec_gpu.begin(), vec_gpu.end(), 10, 10);

    // Calculando a soma dos elementos
    int soma = thrust::reduce(vec_gpu.begin(), vec_gpu.end(), 0, thrust::plus<int>());

    std::cout << "Soma dos elementos: " << soma << std::endl; // Saída: 150

    return 0;
}

Writing soma.cu


DIFERENÇAS PONTO A PONTO

In [12]:
%%writefile diferencas.cu
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <iostream>

int main() {
    // Criando vetores de preços na GPU
    thrust::device_vector<double> AAPL(5, 150.0);  // Exemplo com preço fixo
    thrust::device_vector<double> MSFT(5, 140.0);  // Exemplo com preço fixo
    thrust::device_vector<double> diff(5);         // Para armazenar as diferenças

    // Calculando a diferença ponto a ponto entre os preços
    thrust::transform(AAPL.begin(), AAPL.end(), MSFT.begin(), diff.begin(), thrust::minus<double>());

    // Exibindo as diferenças
    for (int i = 0; i < diff.size(); i++) {
        std::cout << "Diferença " << i << ": " << diff[i] << std::endl;
    }

    return 0;
}

Writing diferencas.cu


## Exercício 1 e 2

### Exercício 1

- Leia os preços das ações de stocks-google.txt e armazenem em um host_vector.
- Crie um device_vector e transfiram os dados do host_vector para ele.
- Meça o tempo de alocação e cópia dos dados usando a biblioteca std::chrono e imprimam esse tempo para ver quanto tempo a operação leva.

### Exercício 2

- Calcule o preço médio das ações do Google nos últimos 10 anos.
- Calcule o preço médio das ações nos últimos 365 dias.
- Encontre o maior e o menor preço do período total e do último ano.



In [13]:
%%writefile stocksEx1e2.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/extrema.h>
#include <iostream>
#include <chrono>
using namespace std;

int main() {
    std::chrono::duration<double> diff;

    // Leitura
    auto leitura_i = std::chrono::steady_clock::now();
    thrust::host_vector<double> host(3112, 0);
    for (int i = 0; i < 3112; i++) {
        cin >> host[i];
    }
    auto leitura_f = std::chrono::steady_clock::now();
    diff = leitura_f - leitura_i;
    cout << "Tempo de LEITURA (em segundos)  " << diff.count() << endl;

    // Copia
    auto copia_i = std::chrono::steady_clock::now();
    thrust::device_vector<double> dev(host);
    auto copia_f = std::chrono::steady_clock::now();
    diff = copia_f - copia_i;
    cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << endl;


    // Calculando o preço médio dos últimos 10 anos
    double soma = thrust::reduce(dev.begin(), dev.end(), 0.0, thrust::plus<double>());
    double preco_medio = soma / 3112;
    cout << "Preço médio 10 anos: " << preco_medio << endl;

    // Calculando o preço médio dos últimos 365 dias
    thrust::device_vector<double> dev_ano(dev.end() - 365, dev.end());
    double ano_soma = thrust::reduce(dev_ano.begin(), dev_ano.end(), 0.0, thrust::plus<double>());
    double ano_media = ano_soma / 365;
    cout << "Preço médio 365 dias: " << ano_media << endl;

    // Encontrando o maior e o menor preço no período total de 10 anos
    auto min_max_total = thrust::minmax_element(dev.begin(), dev.end());
    cout << "Menor preço 10 anos: " << *min_max_total.first << endl;
    cout << "Maior preço 10 anos: " << *min_max_total.second << endl;

    // Encontrando o maior e o menor preço nos últimos 365 dias
    auto min_max_ano = thrust::minmax_element(dev_ano.begin(), dev_ano.end());
    cout << "Menor preço 365 dias: " << *min_max_ano.first << endl;
    cout << "Maior preço 365 dias: " << *min_max_ano.second << endl;

    return 0;
}

Writing stocksEx1e2.cu


In [14]:
!nvcc -arch=sm_70 -std=c++14 stocksEx1e2.cu -o stocksEx1e2

In [15]:
!./stocksEx1e2 < stocks-google.txt

Tempo de LEITURA (em segundos)  0.00432368
Tempo de CÓPIA (em segundos)  0.131204
Preço médio 10 anos: 1580.08
Preço médio 365 dias: 1615.81
Menor preço 10 anos: 1015.24
Maior preço 10 anos: 2200
Menor preço 365 dias: 1060.5
Maior preço 365 dias: 2200


## Exercício 3

- Leia os preços das ações de ambas as empresas e armazenem em dois device_vectors.
- Calcule a diferença ponto a ponto entre os dois vetores e armazenem as diferenças em um terceiro vetor.
- Calcule a média das diferenças para entender qual empresa teve, em média, um preço maior.

In [16]:
%%writefile stocksEx3.cu
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <iostream>
#include <sstream>
#include <string>
using namespace std;

int main() {
    thrust::device_vector<double> AAPL(3520, 0.0);
    thrust::device_vector<double> MSFT(3520, 0.0);
    thrust::device_vector<double> diff(3520);

    string line;
    double value1 = 0.0, value2 = 0.0;
    for (int i = 0; i < 3520; i++) {
        getline(cin, line);
        stringstream ss(line);
        string temp;
        getline(ss, temp, ',');
        value1 = stod(temp);
        getline(ss, temp, ',');
        value2 = stod(temp);
        AAPL[i] = value1;
        MSFT[i] = value2;
    }

    thrust::transform(AAPL.begin(), AAPL.end(), MSFT.begin(), diff.begin(), thrust::minus<double>());

    double soma = thrust::reduce(diff.begin(), diff.end(), 0.0, thrust::plus<double>());
    double media = soma / 3520;
    cout << "Média das diferenças: " << media << endl;

    return 0;
}

Writing stocksEx3.cu


In [17]:
!nvcc -arch=sm_70 -std=c++14 stocksEx3.cu -o stocksEx3

In [18]:
!./stocksEx3 < stocks2.txt

Média das diferenças: -265.232
